In [539]:
import csv
import datetime
import operator
import os
import re
from random import randint
import networkx as nx
import csv
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from operator import itemgetter
from shapely.wkt import loads
import copy
from os import path
import netwulf as nw
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mtick
from os import listdir
import time

In [540]:
NOT_INFECTED=0
INFECTED=1
NUM_NODOS=1
PERCENTAGE = 0.60
pathFile = "/Users/silviadelatorre/Desktop/TFG/PFG/Results/PROPAGACIÓN"

enlaces_global = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_GrafoGlobal.csv"
enlaces_duero = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO DUERO.csv"
enlaces_tajo = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO TAJO.csv"
enlaces_guadiana = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO GUADIANA.csv"
enlaces_jucar = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO JUCAR.csv"
enlaces_ebro = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO EBRO.csv"
enlaces_segura = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO SEGURA.csv"
enlaces_garona = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO GARONA.csv"
enlaces_barbate = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO BARBATE.csv"
enlaces_ter =  "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIU TER.csv"
enlaces_palancia = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIU PALANCIA.csv"
enlaces_guadalquivir = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO GUADALQUIVIR.csv"
rios = [(enlaces_guadalquivir,"RIO GUADALQUIVIR"),(enlaces_palancia,"RIO PALANCIA"),(enlaces_ter,"RIO TER"),(enlaces_barbate,"RIO BARBATE"),(enlaces_garona,"RIO GARONA"),(enlaces_segura,"RIO SEGURA"),(enlaces_guadiana,"RIO GUADIANA"),(enlaces_ebro,"RIO EBRO"),(enlaces_jucar,"RIO JUCAR"),(enlaces_tajo,"RIO TAJO")]

fosfato = "Results/VALORES CONTAMINACION/Fosfato_merged.csv"
amonio = "Results/VALORES CONTAMINACION/Amonio_merged.csv"
fosforo = "Results/VALORES CONTAMINACION/Fosforo_merged.csv"
fitobentos = "Results/VALORES CONTAMINACION/Fitobentos_merged.csv"
nitrato = "Results/VALORES CONTAMINACION/Nitrato_merged.csv"
grado_trofico = "Results/VALORES CONTAMINACION/Grado Trofico_merged.csv"


In [541]:

def CrearGrafo(edge_list):
    # Crear un grafo no dirigido
    G = nx.DiGraph()

    # Analizar el edge list y agregar los nodos y las aristas al grafo
    with open(edge_list, 'r') as f:
        for line in f:
            nodos_y_atributos = line.strip().split(';')
            nodo1_coords = tuple(map(float, nodos_y_atributos[0][1:-1].split(', ')))
            nodo2_coords = tuple(map(float, nodos_y_atributos[1][1:-1].split(', ')))

            # Agregar la arista al grafo
            G.add_edge(nodo2_coords, nodo1_coords)

    print("Número de nodos:", G.number_of_nodes())
    print("Número de aristas:", G.number_of_edges())
    degrees = G.degree()
    nodes = G.nodes()
    n_color = np.asarray([degrees[n] for n in nodes])
    #GraficarRed(G,n_color)
    return G

In [542]:

def StatusNodes(V):
    'Inicializar dataframe de nodos y estados de infeccion'
    global df, df
    
    Nodes=V.nodes()
    df = pd.DataFrame(Nodes)
    #print(df)
    data = {'Node': Nodes, 'Status': [NOT_INFECTED] * len(Nodes)}
    df = pd.DataFrame(data)

    #print(f'Status data frame: {df}')

In [543]:
def Infected_Nodes(V):
    global df

    #StatusNodes(V)
    TotalNodes = len(V.nodes())
    TotalNodesInfect = df['Status'].sum()

    Rate = TotalNodesInfect/TotalNodes
    return Rate


In [544]:
def CodigodelRio(rio):
    if(rio=='RIO GUADALQUIVIR'):
        codigo = '10094'
    elif(rio=='RIO GARONA'):
        codigo = '10034'
    elif(rio=='RIO MIÑO'):
        codigo = '10038'
    elif(rio=='RIO DUERO'):
        codigo = '1004'
    elif(rio=='RIO TAJO'):
        codigo = '1006'
    elif(rio=='RIO GUADIANA'):
        codigo = '1008'
    elif(rio=='RIO BARBATE'):
        codigo = '10098'
    elif(rio=='RI0 TER'):
        codigo = '20054'
    elif(rio=='RIO LLOBREGAT'):
        codigo = '20052'
    elif(rio=='RIO PALANCIA'):
        codigo = '20036'
    elif(rio=='RIO SEGURA'):
        codigo = '20016'
    elif(rio=='RIO EBRO'):
        codigo='2004'
    elif(rio=='RIO JUCAR'):
        codigo='2002'
    
    return codigo

In [545]:
def ObtenerPrimerNodo(G,agente,rio):
    max = 0
    coord_max_fosfato = None
    print(f'Obtener Nodo más contaminado del {rio}')
    codigo = CodigodelRio(rio)
    print(f"El codigo del {rio} es {codigo}")
    with open(agente, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            # Obtiene el ID del río, la coordenada y el valor de fosfato
            id_rio = row[0]
            coord = row[1]
            valor_contaminante = float(row[2])
            # Comprueba si el valor actual del agente es mayor que el máximo
            if str(id_rio).startswith(str(codigo)) and valor_contaminante > max:
                max = valor_contaminante
                coord_max = coord
                
    print("Mayor valor de contaminación es",max)
    return coord_max

In [546]:
def DibujarVariasGraficas(Dir,pC, pR):
    
    #mi_path = os.getcwd() + "\\" + "..\\datos\\" + str(Dir)
    mi_path=Dir
    
    fig = plt.figure()
    ax = plt.subplot(111)
    plt.title('Failure propagation')
    ax.set_xlabel('n')
    ax.set_ylabel('Infection rate')
    fmt = '%.5f' # Format you want the ticks, e.g. '40%'
    xticks = mtick.FormatStrFormatter(fmt)
    ax.yaxis.set_major_formatter(xticks)

    datosHeader = ['iter', 'rate']
    for NomFich in listdir(mi_path):
        pathFile = mi_path + "\\" + NomFich
        datos = pd.read_table(pathFile, 'engine=python', delimiter=' ', header=0, encoding = "ISO-8859-1", names=datosHeader)
        ax.plot(datos['iter'], datos['rate'], marker='o', label='pI: '+str(pC) + ' pR: '+str(pR))
        ax.legend()
    fileFig="TodosGraficos.png"
    pathFileFig = mi_path + "\\" + fileFig
    fig.savefig(pathFileFig)

In [547]:
def ComprobarInfeccion (N, p):
    
    if (np.random.binomial(N,p) > 0):
        return INFECTED
    else:
        return NOT_INFECTED

def ComprobarRecuperacion (p):
    
    if (np.random.binomial(1,p) > 0):
        return NOT_INFECTED
    else:
        return INFECTED 

In [548]:
def VerificarInfeccionVecinos(V,i):

    global TotalNodes, pC

    Neighbours = list(V.successors(i))
    print("NEIGHBOURS",Neighbours)
    NumNodosInfect = df.Status[(df['Node'].isin(Neighbours)) & (df['Status'] == INFECTED)].sum()
    #df.Status[df.Node == i] = ComprobarInfeccion(NumNodosInfect, pC)
    df.loc[df.Node == i, 'Status'] = ComprobarInfeccion(NumNodosInfect, pC)


def AplicarRecuperacion(i):

    global df, pR
	
    df.Status[(df['Node']==i)]=ComprobarRecuperacion(pR)

In [549]:
def GuardarResultados(Time,Rate,totalNodes,iter,path,rio):
    global pathFile, pC, pR

    # Crear un DataFrame con los datos de tiempo y tasa de infección
    datos = pd.DataFrame([[iter,Time, Rate,totalNodes,pC,pR]], columns=['Iteracion','Tiempo Infección', 'Ratio infectados','Nodos Totales','pC','pR'])

    csv_path = os.path.join(path, f'{rio}.csv')
    if not os.path.exists(csv_path):
        # Si el archivo no existe, escribe la cabecera
        datos.to_csv(csv_path, header=True, index=None, mode='w', sep=' ', decimal='.')
    else:
        # Si el archivo ya existe, añade los nuevos datos sin la cabecera
        datos.to_csv(csv_path, header=False, index=None, mode='a', sep=' ', decimal='.')

    Time += 1

In [550]:
def InfectarVecinos(V,iter,path,nombre_rio):
    global Rate, TotalNodes, df
    
    Time = time.time()

    #while Rate < PERCENTAGE:
    # Obtener todos los nodos infectados en este momento
    nodos_infectados = df[df['Status'] == INFECTED]['Node'].values
    print("Nodos infectados",nodos_infectados)
    # Propagar infección a los vecinos de los nodos infectados
    for nodo in nodos_infectados:
        sucesores = list(V.successors(nodo))  # Obtener los sucesores del nodo infectado
        for sucesor in sucesores:
            # Verificar si el sucesor aún no está infectado y propagar la infección con una cierta probabilidad
            NumNodosInfect= df.Status[(df['Node'].isin(sucesores)) & (df['Status']==INFECTED)].sum()
            if df.loc[df['Node'] == sucesor, 'Status'].values[0] == NOT_INFECTED:
                df.loc[df['Node'] == sucesor, 'Status'] = ComprobarInfeccion(NumNodosInfect,pC)
    
    # Calcular el nuevo total de nodos infectados
    TotalNodesInfect = df['Status'][df['Status'] == INFECTED].sum()

    Rate = TotalNodesInfect / TotalNodes

    GuardarResultados(Time,Rate,TotalNodes,iter,path,nombre_rio)
    
    print("Infección alcanzó el " + str(Rate * 100) + "% de los nodos INFECTEDs.")
    print("End time " + str(Time))

In [551]:
def EjecutarIteracionInfeccion(V,Tipo,nombre,path):
    '''Función para ejecutar una iteración de la simulación'''

    global pathFile, Time, df, Rate, TotalNodes,NDir
    
    Iter=0
    print("Iteracion " + str(Iter))
    
    print("Propagando infeccion")

    Nodes=V.nodes()
    TotalNodes = len(Nodes)
    #print("Total nodes " + str(TotalNodes))
    FirstNode = ObtenerPrimerNodo (V, Tipo,nombre)
    print("Primer nodo infectado")
    print(FirstNode)
    FirstNode = tuple(map(float, FirstNode.strip('()').split(', ')))
    df.loc[df['Node'] == FirstNode, 'Status'] = INFECTED

    df.to_csv('nodos_dataframe.csv', index=False)
    
    TotalNodesInfect = df.Status[df.Status==INFECTED].sum()
    print("Total nodos infectados:",TotalNodesInfect)
    
    Rate = Infected_Nodes(V)
    print(f"Ratio infectados INICIAL: {Rate * 100:.4f}%")
	
    start_time = time.time()
    print(f'{Rate} vs {PERCENTAGE}')
    while(Rate < PERCENTAGE):
        Iter += 1
        Rate = InfectarVecinos(V,Iter,path,nombre)
        #EjecutarPropagacionInfeccionTime(V,Rate,start_time)
        print("Ratio:",Rate)
    
    end_time = time.time()  # Registra el tiempo de finalización
    execution_time = end_time - start_time  # Calcula el tiempo de ejecución

    print(f"Ratio FINAL nodos infectados: {Rate * 100:.4f}%")
    print("End time "+str(execution_time))

    

In [552]:
def CrearDirectorio(Tipo):
    print("Creando directorio")
    NDir= "/Users/silviadelatorre/Desktop/TFG/PFG/Results/PROPAGACIÓN/Prop"+Tipo
    os.makedirs(NDir, exist_ok=True)
    return NDir
    

In [553]:

def PropagarInfeccion2 (pR, pC, Tipo,V,rio):

    global NDir
    
    print("Ejecutando iteraciones")
    
    if (Tipo == 'Am'):
        path = CrearDirectorio('Amonio')
        EjecutarIteracionInfeccion(V,amonio,rio,path)
    elif (Tipo == 'Ni'):
        path = CrearDirectorio('Nitrato')
        EjecutarIteracionInfeccion(V,nitrato,rio,path)
    elif (Tipo == 'Fr'):
        path =  CrearDirectorio('Fosforo')
        EjecutarIteracionInfeccion(V,fosforo,rio,path)
    elif (Tipo == 'Ft'):
        path = CrearDirectorio('Fosfato')
        EjecutarIteracionInfeccion(V,fosfato,rio,path)
    elif (Tipo == 'Gt'):
        path = CrearDirectorio('GradoTrofico')
        EjecutarIteracionInfeccion(V,grado_trofico,rio,path)
    else:
        path = CrearDirectorio('Fitobentos')
        EjecutarIteracionInfeccion(V,fitobentos,rio,path)


In [554]:

def PropagarInfeccionTodasProbabilidades(i, Tipo,V,rio):
    global pC, pR
    
    pC=i
    pR=0
    print("Probabilidad contagio " + str(pC))

    PropagarInfeccion2 (pR, pC, Tipo,V,rio)

In [555]:
def Menu2(graph,nombre_rio):
    while True:
        print("Choose an option:")
        print("1. Propagation with infection starting in node with highest nitrate contamination")
        print("2. Propagation with infection starting in node with highest phosphorus contamination")
        print("3. Propagation with infection starting in node with highest phosphate contamination")
        print("4. Propagation with infection starting in node with highest ammonium contamination")
        print("5. Propagation with infection starting in node with highest phytobenthos contamination")
        print("6. Propagation with infection starting in node with highest trophic grade contamination")
        print("7. Propagation with infection starting in node with highest overall contamination")
        print("8. Exit")
        
        StatusNodes(graph)

        option = input("")

        if option == "1":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Nt",graph,nombre_rio)
            break
        elif option == "2":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Fr",graph,nombre_rio)
            break
        elif option == "3":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Ft",graph,nombre_rio)
            break
        elif option == "4":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Am",graph,nombre_rio)
            break
        elif option == "5":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Fit",graph,nombre_rio)
            break
        elif option == "6":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Gt",graph,nombre_rio)
            break
        elif option == "7":
            break
        else:
            print("Invalid option. Please choose again.")

    return option

In [556]:

def Menu1():
    while True:
        print("Choose an option:")
        print("1. Simulate propagation for the global network of rivers")
        print("2. Simulate propagation for each individual non-connected subnetwork")
        print("3. Exit")

        option = input("")
        print("You chose option: ",option)

        if option == "1":
            print("Simulating propagation for the entire network, infecting one node per subnetwork")
            G = CrearGrafo(enlaces_global)
            Menu2(G,"RIOS ESPAÑA")
            return option
        
        elif option == "2":
            print("Simulating propagation for each individual subnetwork")
            
            for enlaces,nombre_rio in rios:
                print(f"Propagar infección en rio {nombre_rio}")
                G = CrearGrafo(enlaces)
                Menu2(G,nombre_rio)
            
            return option
        elif option == "3":
            print("Exiting")
            return option
        else:
            print("Invalid option. Please choose again.")



In [557]:
option1 = Menu1()

Choose an option:
1. Simulate propagation for the global network of rivers
2. Simulate propagation for each individual non-connected subnetwork
3. Exit
You chose option:  2
Simulating propagation for each individual subnetwork
Propagar infección en rio RIO GUADALQUIVIR
Número de nodos: 63815
Número de aristas: 66383
Choose an option:
1. Propagation with infection starting in node with highest nitrate contamination
2. Propagation with infection starting in node with highest phosphorus contamination
3. Propagation with infection starting in node with highest phosphate contamination
4. Propagation with infection starting in node with highest ammonium contamination
5. Propagation with infection starting in node with highest phytobenthos contamination
6. Propagation with infection starting in node with highest trophic grade contamination
7. Propagation with infection starting in node with highest overall contamination
8. Exit
Probabilidad contagio 0.5
Ejecutando iteraciones
Creando director

TypeError: '<' not supported between instances of 'NoneType' and 'float'